In [278]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime

ss = pd.read_csv("sample_submission.csv")
train_df = pd.read_csv("train.csv/train.csv", parse_dates=['timestamp'])
test_df = pd.read_csv("test.csv/test.csv", parse_dates=['timestamp'])
fx = pd.read_excel('BAD_ADDRESS_FIX.xlsx')
macro = pd.read_csv("macro.csv")

#фиксим данные
fx.set_index('id',inplace=True)
train_df.set_index('id',inplace=True)
test_df.set_index('id',inplace=True)

train_df.update(fx)
test_df.update(fx)

print('Fix in train: ', train_df.index.intersection(fx.index).shape[0])
print('Fix in test : ', test_df.index.intersection(fx.index).shape[0])

train_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)
id_test = test_df.id
id_train = train_df.id

('Fix in train: ', 550)
('Fix in test : ', 149)


In [279]:
norm_features = ['id', 'timestamp', 'full_sq', 'life_sq', 'floor','max_floor','build_year', 'sub_area', 'price_doc']
geo_features = ['mkad_km','kremlin_km','ttk_km','sadovoe_km']
norm_features_for_clust = ['id', 'timestamp', 'full_sq', 'life_sq','material','floor','max_floor','build_year', 'price_doc']

In [280]:
train_df[train_df['full_sq'] == train_df['life_sq']].shape

(2750, 292)

In [281]:
# фиксим данные
def fix_full_life_sq(l):
    full,life = l
    if life>full:
        if full>=15 and life - full <= 30:
            return [life, full]
        
        if full>=15 and 10*full > life:
            return [full, (1.*life)/10]
        if full>=15:
            return [full, np.nan]
        
        if full<15 and life> 15 and life< 150:
            return [life, np.nan]
        return [np.nan, np.nan]
    else:
        if life<15:
            return [full, np.nan]
        
        return [full, life]
#train
train_df.loc[:][['full_sq','life_sq']] = map(fix_full_life_sq,train_df[['full_sq','life_sq']].values)
#test
test_df.loc[:][['full_sq','life_sq']] = map(fix_full_life_sq,test_df[['full_sq','life_sq']].values)

def fix_state(state):
    if state == 33:
        return 3
    else:
        return state
    
#в тесте все ок
train_df.loc[:]['state'] = map(fix_state,train_df['state'].values)

def fix_floor(l):
    floor, max_floor = l
    if max_floor<floor:
        if max_floor == 0:
            return [floor, np.nan]
        else:
        #тут могли поменять местами
            return [floor, np.nan]
    else:
        return [floor, max_floor]
    
#train
train_df.loc[:][['floor','max_floor']] = map(fix_floor,train_df[['floor','max_floor']].values)
#test
test_df.loc[:][['floor','max_floor']] = map(fix_floor,test_df[['floor','max_floor']].values)

def fix_year(year):
    if year<1800 or year>2030:
        return np.nan
    else:
        return year
#train
train_df.loc[:]['build_year'] = map(fix_year,train_df['build_year'].values)
#test
test_df.loc[:]['build_year'] = map(fix_year,test_df['build_year'].values)    


def fix_num_room(num_room):
    if num_room == 0:
        return np.nan
    else:
        return num_room
    
#train
train_df.loc[:]['num_room'] = map(fix_num_room,train_df['num_room'].values)
#test
test_df.loc[:]['num_room'] = map(fix_num_room,test_df['num_room'].values) 

In [282]:
# добавим временные признаки
train_df['month'] = train_df.timestamp.dt.month
train_df['year'] = train_df.timestamp.dt.year

test_df['month'] = test_df.timestamp.dt.month
test_df['year'] = test_df.timestamp.dt.year

In [387]:
data_df = train_df.copy()
#data_df.loc[:][tf[:-1]] = data_df[tf[:-1]].round(7)

grouped_df = data_df.groupby(geo_features+['sub_area']).count().sort_values('id',ascending=False)
#grouped_df[grouped_df['id']>3][norm_features_for_clust].sum()

tf = geo_features+['sub_area']

In [389]:
#количество NaN'ов
#print 'total: %d' % train_df.shape[0]
#pd.DataFrame(train_df[norm_features_for_clust+geo_features].isnull().sum(), columns=['train'])

In [390]:
#усредняем по геогруппам + общий material и max_floor
temp_df = data_df.copy()

#находит моду внутри кластера
temp_build_year = temp_df[~temp_df['build_year'].isnull()][tf+['material', 'max_floor','build_year']]\
    .groupby(tf+['material', 'max_floor']).agg(lambda x:x.value_counts().index[0])
    
temp_df.set_index(tf+['material','max_floor'],inplace=True)

def fix_clust_build_year(l):
    build_year, clust_build_year = l
    if np.isnan(build_year):
        return clust_build_year
    else:
        return build_year
    
clust_build_df = temp_df.join(temp_build_year, how='left', rsuffix='_mode')
clust_build_df.reset_index(inplace=True)

#train
clust_build_df['clust_build_year'] = map(fix_clust_build_year,\
                                                clust_build_df[['build_year', 'build_year_mode']].values)
clust_build_df.drop('build_year_mode', axis=1, inplace=True)

print clust_build_df['build_year'].count()
clust_build_df['build_year'] = clust_build_df['clust_build_year']
clust_build_df.drop('clust_build_year',axis=1,inplace=True)
print clust_build_df['build_year'].count()
data_df = clust_build_df.copy()
del(temp_df, temp_build_year,clust_build_df)

15960
18751


In [1]:
data_df.set_index(tf).loc[grouped_df.index[5]][norm_features_for_clust].count()

NameError: name 'data_df' is not defined

In [410]:
train_df.set_index(tf).loc[grouped_df.index[5]][norm_features_for_clust].count()

/home/kondrat/anaconda2/envs/theano/lib/python2.7/site-packages/ipykernel/__main__.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  if __name__ == '__main__':


id            282
timestamp     282
full_sq       282
life_sq       235
material       62
floor         265
max_floor      43
build_year     35
price_doc     282
dtype: int64

In [81]:
# удалим заведамо запредельные цены
train_df = train_df[1.*train_df.price_doc/train_df.full_sq <= 700000]
train_df = train_df[1.*train_df.price_doc/train_df.full_sq >= 70000]

# добавим месяц-год
month_year = (train_df.timestamp.dt.month + train_df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
train_df['month_year_cnt'] = month_year.map(month_year_cnt_map)

month_year = (test_df.timestamp.dt.month + test_df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
test_df['month_year_cnt'] = month_year.map(month_year_cnt_map)

# добавим неделю-год
week_year = (train_df.timestamp.dt.weekofyear + train_df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
train_df['week_year_cnt'] = week_year.map(week_year_cnt_map)

week_year = (test_df.timestamp.dt.weekofyear + test_df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
test_df['week_year_cnt'] = week_year.map(week_year_cnt_map)

# добавим месяц
train_df['month'] = train_df.timestamp.dt.month
#train_df['dow'] = train_df.timestamp.dt.dayofweek

test_df['month'] = test_df.timestamp.dt.month
#test_df['dow'] = test_df.timestamp.dt.dayofweek

In [83]:
y_train=np.log(my_train.price_doc)
x_train = my_train.drop(["timestamp", "price_doc"], axis=1)
x_test = my_test.drop(["timestamp"], axis=1)

x_train.set_index('id', inplace=True)
x_test.set_index('id', inplace=True)

train_idx = x_train.index
test_idx = x_test.index

temp_df = pd.concat([x_train, x_test])

for c in temp_df.columns:
    if temp_df[c].dtype == 'object':
        none_idx = temp_df[temp_df[c].isnull()].index
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(temp_df[c].values)) 
        temp_df[c] = lbl.transform(list(temp_df[c].values))
        #x_train.drop(c,axis=1,inplace=True)
        temp_df[c].loc[none_idx] = np.nan

x_train = temp_df.loc[train_idx]
x_test = temp_df.loc[test_idx]

x_train.reset_index(inplace=True)
x_test.reset_index(inplace=True)

x_train.drop(['id'], axis=1, inplace=True)
x_test.drop(['id'], axis=1, inplace=True)
        
#x_train.fillna(0,inplace=True)

#x_test.fillna(0,inplace=True)

xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000,
    verbose_eval=50, show_stdv=False)


num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

y_predict = np.exp(model.predict(dtest))
#y_predict = np.round(y_predict * 0.99)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})

output.to_csv('my_pred.csv', index=False)

(27140, 299)
(7662, 298)
[0]	train-rmse:8.58903e+06	test-rmse:8.59196e+06
[50]	train-rmse:1.7184e+06	test-rmse:2.13011e+06
[100]	train-rmse:1.3003e+06	test-rmse:1.86744e+06
[150]	train-rmse:1.18517e+06	test-rmse:1.82076e+06
[200]	train-rmse:1.1039e+06	test-rmse:1.79548e+06
[250]	train-rmse:1.03422e+06	test-rmse:1.7762e+06
[300]	train-rmse:980267	test-rmse:1.76287e+06
[350]	train-rmse:937359	test-rmse:1.75512e+06
[400]	train-rmse:898811	test-rmse:1.74982e+06
[450]	train-rmse:867282	test-rmse:1.74584e+06
[500]	train-rmse:838469	test-rmse:1.74265e+06
[550]	train-rmse:811489	test-rmse:1.73945e+06
[600]	train-rmse:785822	test-rmse:1.73653e+06
[650]	train-rmse:763540	test-rmse:1.73412e+06
[700]	train-rmse:743422	test-rmse:1.73294e+06
[750]	train-rmse:724485	test-rmse:1.73226e+06
[800]	train-rmse:705897	test-rmse:1.7313e+06
[850]	train-rmse:688627	test-rmse:1.72972e+06
[900]	train-rmse:672701	test-rmse:1.72958e+06
[950]	train-rmse:656412	test-rmse:1.72872e+06


In [84]:
output

,id,price_doc
0,30474,6242745.50
1,30475,8657184.00
2,30476,6389490.00
3,30477,6431807.50
4,30478,5284613.50
5,30479,9626468.00
6,30480,4515084.00
7,30481,4278081.50
8,30482,5654755.00
9,30483,5371272.00
